ChEn-3170: Computational Methods in Chemical Engineering Fall 2018 UMass Lowell; Prof. V. F. de Almeida **11Oct2018**

# 07. Computational Stoichiometry
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \DeclareMathOperator{\rank}{rank}
$

---
## Table of Contents
* [Introduction](#intro)
* [Stoichiometric matrix](#stoicmtrx)
* [Linear independent](#indepen)
* [Full-rank, sub-reaction mechanism](#subreact)
* [Reaction frequency](#rxnfreq)
---

## Introduction<a id="intro"></a>
"Stoichiometry is essentially the bookkeeping of the material components of a chemical system." Rutherford Aris in Elementary Chemical Reactor Analysis. Computational stoichiometry is the matrix analysis of the stoichiometric matrix of a chemical reaction mechanism. We will use it for two purposes: 
 + Evaluate sets of independent reactions
 + Obtain basic insight on reaction mechanisms from a linear algebra standpoint
 + Evaluate reaction rates and species production rates

Recall [linear algebra notes.](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/ErfDAD_jL1tHkDrx29w89-MBcOgK4JAnEYSheRoxkL_0sw?e=fpzG5K)

[Notes](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/Evb2l8y2WNJCgNvJhcF0Pc4B-_TOOflJkiEAgCfICZwNVA?e=sV9YK0) on computational stoichiometry including an introduction to the linear, full-rank, least-squares method.

## Stoichiometric matrix<a id="stoicmtrx"></a>
After reading a reaction mechanism from file, and storing the input data into data types, construct the stoichiometric coefficient matrix.

In [ ]:
'''Open file for an ammonia oxidation reaction mechanism'''

# open file in reading mode 'r' (default), text 't' (default)
finput = open('data/ammonia-rxn.txt','rt')

!cat 'data/ammonia-rxn.txt'


In [ ]:
'''Build the reactions list'''

reactions = list()
for line in finput:
    reactions.append( line.strip() )
for r in reactions: 
    i = reactions.index(r)
    print('r%s'%i,': ',r)

finput.close()

In [ ]:
'''Shuffle the order of reactions to avoid any bias'''

import random
random.shuffle( reactions )
for r in reactions: 
    i = reactions.index(r)
    print('r%s'%i,': ',r)

In [ ]:
'''Create the species list'''

species_tmp = list()  # temporary list for species
for r in reactions:
    left = r.split('<')
    right = r.split('>')
    species_left = left[0].split('+')
    species_right = right[1].split('+')
    species_rxn = species_left + species_right
    print('species_rxn =',species_rxn)
    for i in species_rxn:
        species_tmp.append( i.split(' ')[1] )    
print('\nspecies_tmp =',species_tmp)

species_filter = set(species_tmp) # filter species as a set

species = list( species_filter )  # convert species set to list 
print('\nspecies =\n',species)
print('# of species =',len(species))

In [ ]:
'''Create the stoichiometric matrix'''

import numpy as np
s_mtrx = np.zeros((len(reactions),len(species)))
for r in reactions:
    left = r.split('<')
    left_terms = left[0].split('+')
    for t in left_terms:
        coeff = float(t.split(' ')[0])
        i_row = reactions.index(r)
        species_member = t.split(' ')[1]
        j_col = species.index(species_member)
        s_mtrx[i_row,j_col] = -1.0 * coeff
        
    right = r.split('>')
    right_terms = right[1].split('+')
    for t in right_terms:
        coeff = float(t.split(' ')[0])
        i_row = reactions.index(r)
        species_member = t.split(' ')[1]
        j_col = species.index(species_member)
        s_mtrx[i_row,j_col] = 1.0 * coeff
        
print('species',species)
print('s_mtrx =\n',s_mtrx)
print('m x n =',s_mtrx.shape)

## Linearly independent reactions<a id="indepen"></a>
The stoichiometric matrix represents a linear system of equations in two different instances. First, a mass balance when the molar masses of the species involved are used as a vector. The product of the stoichiometric matrix and this vector must result in the zero vector. Second, the reaction rates are related to the species production rates in a similar way. In both cases the system of equations lead to a rectangular system. Here we compute the rank of the stoichiomtric matrix to discover how many of the reactions are linearly independent.

In [ ]:
'''How many reactions are independent?'''

from chen_3170.toolkit import lu_factorization

# using complete pivoting
(p_mtrx, q_mtrx, l_mtrx, u_mtrx, s_rank) = lu_factorization( s_mtrx, 'complete', pivot_tol=1e-8 )

print('my rank =',s_rank)
print('numpy rank = ',np.linalg.matrix_rank( s_mtrx, tol=1e-8 ))
np.set_printoptions(precision=2)
print('u_mtrx =\n',u_mtrx)

In [ ]:
'''How many reactions are independent?'''

# partial pivoting could fail
(p_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx, 'partial', pivot_tol=1e-8 )

print('my rank =',rank)
print('u_mtrx =\n',u_mtrx)

In [ ]:
'''How many reactions are independent? Let's break partial pivoting'''

# partial pivoting could fail; try all cases of reaction permutation
import math
import itertools
rxn_permutations = list( itertools.permutations(range(len(reactions))) )
print('# of permutations = ', len(rxn_permutations))
print('# of reactions!   = ',math.factorial(len(reactions)))

print(s_mtrx)
print(rxn_permutations[1200])
print(s_mtrx[rxn_permutations[1200],:])
#print(s_mtrx[[0,5,2,1,4,3,6],:])

for perm in rxn_permutations:
    (p_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx[perm,:], 'partial', pivot_tol=1e-8 )    
    assert rank == 3

print('done')

In [ ]:
'''How many reactions are independent?'''

# no pivoting will fail
(p_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx, pivot_tol=1e-8 )

## Full-rank, sub-reaction mechanisms<a id="subreact"></a>
Here we form all possible combination of full-rank sub-reaction mechanisms.

In [ ]:
'''Total number of rank-reaction sets'''

# n_reactions choose s_rank binomial formula
import math
print('# of binomial terms =',
      math.factorial(n_reactions)/math.factorial(n_reactions-s_rank)/math.factorial(s_rank))

from itertools import combinations
n_reactions = len(reactions)
tmp = combinations(range(n_reactions),s_rank)
reaction_sets = [i for i in tmp]

print(reaction_sets)

In [ ]:
'''Finding sets of linearly independent reactions'''

sub_reactions = list()
for r in reaction_sets:
    s_mtrx_k = s_mtrx[r,:]
    (p_mtrx, q_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx_k, 'complete', pivot_tol=1e-8 )
#   rank = np.linalg.matrix_rank( s_mtrx_k, tol=1e-8 )
    if rank == s_rank:
        sub_reactions.append( [r, [reactions[i] for i in r]] )  # list elements

print('# of sub_reactions =',len(sub_reactions))        
print(sub_reactions)

for s in sub_reactions:
    print('Linearly Independent Reaction Set %s'%sub_reactions.index(s))
    for (i,r) in zip(s[0],s[1]):
        print('r%s'%i,r)

## Reaction frequency analysis<a id="rxnfreq"></a>


In [ ]:
'''How often a reaction appears on a reaction sub-mechanisms'''

reactions_hits = np.zeros(n_reactions)
for s in sub_reactions:
    for i in s[0]:
        reactions_hits[i] += 1
        
print(reactions_hits/len(sub_reactions))    

In [ ]:
'''Plot the frequency of appearance of reactions in sub-mechanisms'''

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 4]

fig, ax = plt.subplots(figsize=(20,6))
sort_results = sorted( zip(reactions,reactions_hits/len(sub_reactions)*100), key=lambda entry: entry[1], reverse=True )
reactions_sorted = [a for (a,b) in sort_results]
hits_sorted = [b for (a,b) in sort_results]
ax.bar(reactions_sorted, hits_sorted,color='orange')
for r in reactions_sorted:
    idx = reactions.index(r)
    i = reactions_sorted.index(r)
    reactions_sorted[i] = 'r'+str(idx)+': '+ r
plt.xticks(range(len(reactions)),reactions_sorted,rotation=60,fontsize=14)
ax.set_ylabel('Frequency [%]',fontsize=16)
ax.xaxis.grid(True,linestyle='-',which='major',color='lightgrey',alpha=0.9)
fig.suptitle('Reaction Appearance Frequency Analysis for Rank = '+str(s_rank),fontsize=20)
plt.show()

In [ ]:
'''Sorting reaction sub-mechanisms based on frequency'''

sub_reactions_score = list()
for s in sub_reactions:
    score = 0
    for i in s[0]:
        score += reactions_hits[i]
    sub_reactions_score.append( score )

sub_reactions_score = np.array(sub_reactions_score) 
sub_reactions_score /= sub_reactions_score.max()
sub_reactions_score *= 10.0


results = sorted( zip(sub_reactions,sub_reactions_score), key=lambda entry: entry[1], reverse=True )

sub_reactions       = [a for (a,b) in results]
sub_reactions_score = [b for (a,b) in results]

# encode score in to sub_reactions mech. data structure
for (sr,score) in zip(sub_reactions,sub_reactions_score):
    sr += [score]

In [ ]:
'''Principal reaction mechanisms'''

max_score = max([sr[2] for sr in sub_reactions])

for sr  in sub_reactions:
    if sr[2] < max_score: continue
    print('Linearly Independent Reaction Set %s score %s'%(sub_reactions.index(sr),sr[2]))
    for (i,r) in zip(sr[0],sr[1]):
        print('r%s'%i,r)

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
ax.bar(range(len(sub_reactions)), [sr[2] for sr in sub_reactions],color='green')
plt.xticks(range(len(sub_reactions)),[sr[0] for sr in sub_reactions],rotation=60,fontsize=14)
ax.set_ylabel('Score [1->10]',fontsize=16)
ax.set_xlabel('Sub-Reaction Mechanisms',fontsize=16)
ax.xaxis.grid(True,linestyle='-',which='major',color='lightgrey',alpha=0.9)
fig.suptitle('Scoring of Sub-Reaction Mechanisms for Rank = '+str(s_rank),fontsize=20)
plt.show()